In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random as rn
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
path =   # Enter the directory of your project folder
PATH = path + "/Results"

if not os.path.exists(PATH):
    os.mkdir(PATH) 

In [ ]:
df = pd.read_csv(path+"//ToyDataset.csv").drop(['Unnamed: 0'],axis=1)
df1 = pd.read_csv(PATH+"//Data one hot encoded.csv").drop(['Unnamed: 0'],axis=1)
clientID = df.ClientID.unique()

df["EntryDate"] = pd.to_datetime(df['EntryDate'])
df["ExitDate"] = pd.to_datetime(df['ExitDate'])
df = df.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)
df1["EntryDate"] = pd.to_datetime(df1['EntryDate'])
df1["ExitDate"] = pd.to_datetime(df1['ExitDate'])
df1 = df1.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)

In [ ]:
goal = [10,11]
cycle_pd = pd.DataFrame(columns = ['ClientID','length','projectType','target'])
index = {}
for i in range(len(clientID)):
    closed = []
    temp = df[df['ClientID']==clientID[i]].reset_index(drop=True)
    for j in range(len(temp)):
        if temp['ProjectType'][j] not in closed:
            closed.append(temp['ProjectType'][j])
            temp1 = temp[temp['ProjectType']==temp['ProjectType'][j]]
            if len(temp1)>2:
                for k in range(1,len(temp1)):
                    length = temp1.index[k]-temp1.index[k-1]
                    if temp.Destination.iloc[-1] in goal: 
                        cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'successful'},ignore_index=True)
                    elif np.isnan(temp.Destination.iloc[-1]):
                        cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'abandoned'},ignore_index=True)
                    else: 
                        cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'unsuccessful'},ignore_index=True)


            elif len(temp1)==2:
                length = temp1.index[-1]-temp1.index[0]
                if temp.Destination.iloc[-1] in goal: 
                    cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'successful'},ignore_index=True)
                elif np.isnan(temp.Destination.iloc[-1]):
                    cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'abandoned'},ignore_index=True)
                else: 
                    cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'unsuccessful'},ignore_index=True)



In [ ]:
cycle_goal = pd.DataFrame(columns=['ClientID',"Target","CycleNumber"])
for i in range(len(clientID)):
    temp = df[df['ClientID']==clientID[i]]
    if temp['Destination'].iloc[-1] in goal:
        if clientID[i] in np.array(cycle_pd['ClientID']):
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'successful',"cycle":'yes',"CycleNumber":len(cycle_pd[cycle_pd["ClientID"]==clientID[i]])},ignore_index=True)
        else:
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'successful',"cycle":'no',"CycleNumber":"None"},ignore_index=True)
    elif np.isnan(temp.Destination.iloc[-1]):
        if clientID[i] in np.array(cycle_pd['ClientID']):
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'abandoned',"cycle":'yes',"CycleNumber":len(cycle_pd[cycle_pd["ClientID"]==clientID[i]])},ignore_index=True)
        else:
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'abandoned',"cycle":'no',"CycleNumber":"None"},ignore_index=True)    
    else:
        if clientID[i] in np.array(cycle_pd['ClientID']):
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'unsuccessful',"cycle":'yes',"CycleNumber":len(cycle_pd[cycle_pd["ClientID"]==clientID[i]])},ignore_index=True)
        else:
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'unsuccessful',"cycle":'no',"CycleNumber":"None"},ignore_index=True)



In [ ]:
cycle_goal.groupby(['Target',"cycle"]).count()

In [ ]:
741/(741+268)

In [ ]:
613/(613+324)

In [ ]:
3356/(3356+709)

In [ ]:
x1 = np.array(cycle_goal[cycle_goal['Target']=='successful'].groupby('CycleNumber').count()["Target"].index,dtype=str)
y1 = np.array(cycle_goal[cycle_goal['Target']=='successful'].groupby('CycleNumber').count()["Target"].values)

x2 = np.array(cycle_goal[cycle_goal['Target']=='unsuccessful'].groupby('CycleNumber').count()["Target"].index,dtype=str)
y2 = np.array(cycle_goal[cycle_goal['Target']=='unsuccessful'].groupby('CycleNumber').count()["Target"].values)

x3 = np.array(cycle_goal[cycle_goal['Target']=='abandoned'].groupby('CycleNumber').count()["Target"].index,dtype=str)
y3 = np.array(cycle_goal[cycle_goal['Target']=='abandoned'].groupby('CycleNumber').count()["Target"].values)



In [ ]:
x1 = np.array(cycle_pd[cycle_pd['target']=='successful'].groupby('length').count()["target"].index,dtype=str)
y1 = np.array(cycle_pd[cycle_pd['target']=='successful'].groupby('length').count()["target"].values)
x2 = np.array(cycle_pd[cycle_pd['target']=='unsuccessful'].groupby('length').count()["target"].index,dtype=str)
y2 = np.array(cycle_pd[cycle_pd['target']=='unsuccessful'].groupby('length').count()["target"].values)
x3 = np.array(cycle_pd[cycle_pd['target']=='abandoned'].groupby('length').count()["target"].index,dtype=str)
y3 = np.array(cycle_pd[cycle_pd['target']=='abandoned'].groupby('length').count()["target"].values)





In [ ]:
fontsize = 25
plt.figure(figsize=(20,10))
width = 0.25
plt.bar(x2.astype(int),y2,color='c',label='unsuccessful',alpha=1,width = 0.25)
plt.bar(x3.astype(int)+width,y3,color='k',label='abandoned',alpha=1,width = 0.25)

plt.bar(x1.astype(int)+width*2,y1,color='orange',label='successful',alpha=1,width = 0.25)
plt.legend(fontsize = fontsize)
# plt.xlabel("Cycle length",fontsize = fontsize)
# plt.ylabel('Frequency',fontsize = fontsize)
plt.xticks(fontsize = fontsize)
plt.yticks(fontsize = fontsize)
plt.yscale('log')
plt.savefig(PATH+"//Cycle_length_frequency.png")



